In [2]:
import pandas as pd

## Vista rápida.
Se leen los datos y luego se toma una muestra de ejemplo para realizar una vista rápida de los mismos y ver la estructura general del set de datos.
Para esto usamos la función head() (o tail()) de pandas, que muestra las 5 primeras (o últimas) filas del data set. También se les puede pasar un número entero N como parámetro y con esto nos mostrará las N primeras (o últimas) filas.

In [3]:
tweets = spotify_songs = pd.read_csv('Data/train.csv')
tweets.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


A simple vista podemos ver que nuestro set de datos cuenta con 5 columnas: id, keyword, location, text, target.\
También podemos ver que las columnas keyword y location pueden ser nulas.

## Tamaño de los datos.
Se analizará la dimensión del dataset: su forma, su tamaño y cuantos valores no nulos contiene cada columna.

In [4]:
tweets.shape

(7613, 5)

In [5]:
tweets.size

38065

In [6]:
tweets.count()

id          7613
keyword     7552
location    5080
text        7613
target      7613
dtype: int64

Como podemos observar solo tenemos 5 columnas y un total de 7613 filas. De las columnas solo 3 contienen todos los datos: id, text, target; mientras que keyword y location presentan valores nulos, como bien está especificado en la descripción del dataset.

## Columnas
Se realizará un análisis de las columnas del set de datos: qué significa cada una, tipo de dato, valores nulos e información estadistica de aquellas columnas que sean numéricas.

In [7]:
tweets.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

Como habíamos visto anteriormente tenemos 5 columnas: id, keyword, location, text y target.\
Leyendo la documentación presentada en la competencia de Kaggle (O también el enunciado del TP) se puede ver de qué trata cada columna:\
Id: Identificador único de cada Tweet.\
Keyword: una palabra clave para cada Tweet (Podría no estar).\
Location: Ubicación desde donde fue enviado el Tweet.\
Text: el texto del tweet.\
Target: indica si se trata de un desastre real (1) o no (0).

In [8]:
tweets.dtypes

id           int64
keyword     object
location    object
text        object
target       int64
dtype: object

Se puede observar que el dataset solo tiene tipos de datos int64 u Object en sus columnas.\
Keyword y text son texto entonces es correcto que sean del tipo object.\
Location será analizado a continuación para decidir si es conveniente realizar algún tipo de transformación sobre su tipo de dato o no.

Otra alternativa para ver las columnas, la cantidad de valores no nulos y el tipo de dato de cada columna del dataframe es hacer un .info() sobre el frame:

In [9]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Veo la cantidad de nulos de location y de keyword:

In [10]:
tweets['location'].isnull().value_counts()

False    5080
True     2533
Name: location, dtype: int64

In [11]:
tweets['keyword'].isnull().value_counts()

False    7552
True       61
Name: keyword, dtype: int64

Se puede observar que las columnas id, text y target tienen todos sus valores no nulos, mientras que location tiene 2533 valores nulos y la columna keyword tiene 61 valores nulos.

Se analiza los valores de cada columna y la cantidad de veces que se repite cada una en el set de datos:

### ID:
Identificador único de cada Tweet.\
Se observó que la columna es del tipo int64.\
En cuanto los valores de id, como todo id, esperamos que sea no nulo y único para cada fila:\

In [12]:
tweets['id'].value_counts()

2047     1
9454     1
1234     1
3283     1
9430     1
        ..
581      1
6726     1
4679     1
10824    1
2049     1
Name: id, Length: 7613, dtype: int64

Podemos ver que hay 7613 valores (Mismo valor que cantidad de filas) diferentes de ids y que cada uno se repite 1 sola vez. Esto es lo esperado para cualquier id.

### Keyword:
Una palabra clave para cada Tweet que puede ser nulo o no.\
Se observó que la columna es del tipo Object.

In [13]:
tweets['keyword'].value_counts()

fatalities               45
deluge                   42
armageddon               42
body%20bags              41
sinking                  41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

Se observa que keyword es una palabra única que actúa como clave para cada tweet.\
La cantidad de palabras claves diferentes que tiene el dataset es de 221.

In [14]:
tweets['keyword'].value_counts().sum()

7552

Se hace la suma de la cantidad de veces que aparece cada keyword en el dataset se observa que esta coincide con la cantidad de valores no nulos de la columna.

### Location.
La ubicación desde donde fue enviado el Tweet.\
Es de tipo de dato Object.

In [15]:
tweets['location'].value_counts()

USA                     104
New York                 71
United States            50
London                   45
Canada                   29
                       ... 
Earth-616                 1
21, Porto                 1
CHICAGO (312)             1
Los Angeles New York      1
Iliff,Colorado            1
Name: location, Length: 3341, dtype: int64

Con esto se entiende por qué es un tipo object: es el nombre de la ciudad y/o país en donde se realizó el tweet.\
Se observa que hay 3341 entradas diferentes para location aunque algunas de ellas no precisamente son una ciudad o un país.\
A su vez, observamos que desde Estados Unidos de América se enviaron la mayor cantidad de tweets a analizar.

In [16]:
tweets['location'].value_counts().sum()

5080

Si se realiza la suma vemos que coincide con la cantidad de valores no nulos observados anteriormente.

### Target
Indica si se trata de un desastre real (1) or no (2).\
Es de tipo int64.

In [17]:
tweets['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

Se observa que target solo puede tomar dos valores: 0 para un desastre falso; 1 para un desastre real.\
La cantidad de desastres falsos es mayor que la cantidad de desastre reales.

### Text
El texto del Tweet.\
Su tipo de dato es object dado que es un texto y es no nulo.

In [18]:
tweets['text'].value_counts()

11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charged with manslaughter over the fatal sh...        10
The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'                               6
He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam                      6
#Bestnaijamade: 16yr old PKK suicide bomber who detonated bomb in ... http://t.co/KSAwlYuX02 bestnaijamade bestnaijamade bestnaijamade beÛ_     6
Madhya Pradesh Train Derailment: Village Youth Saved Many Lives                                                                                  5
                                                                                                                                                ..
Reddit Will Now Quarantine Offensive Content: Reddit co-founder and CEO Steve Huffman has unveiled more specif... http

Se puede observar que la cantidad de tweets diferentes no coincide con la cantidad de filas que tenemos en el dataset, por lo que raramente se observan tweets que están repetidos pero tienen diferentes ids.

In [19]:
tweetRepsDataframe = tweets['text'].value_counts().to_frame()
tweetRepsDataframe.columns = ['reps']
tweetRepsDataframe[tweetRepsDataframe['reps'] > 1].count()

reps    69
dtype: int64

Se observa que la cantidad de tweets con texto repetido es de 69 tweets.

In [20]:
tweetRepsDataframe['reps'].value_counts()

1     7434
2       50
3       10
4        4
6        3
10       1
5        1
Name: reps, dtype: int64

De los 69 tweets repetidos:\
50 se repiten 2 veces;\
10 se repiten 3 veces;\
4 se repiten 4 veces;\
3 se repiten 6 veces;\
1 se repite 10 veces;\
1 se repite 5 veces.

### Uso de memoria
A continuación analizaremos la cantidad de memoria que hace cada columna para ver si es necesario hacer alguna transformación para mejorar el uso de memoria:

In [21]:
print((tweets.memory_usage(deep=False) / (1024 * 1024)))
print((tweets.memory_usage(deep=False) / (1024 * 1024)).sum())

Index       0.000122
id          0.058083
keyword     0.058083
location    0.058083
text        0.058083
target      0.058083
dtype: float64
0.29053497314453125


Se puede observar que el uso de memoria es muy bajo, casi 0,3 MB y no implicará un problema a futuro ya que el set de datos no es un set de gran magnitud, solo tiene 7613 registros.

### Nuevas columnas
Para realizar el análisis del dataset vamos a necesitar de crear nuevas columnas, como así también prescindir de algunas de ellas.

#### Longitud del texto de los tweets.
Una columna que brinda mucha información y es de gran utilidad a la hora de analizar relaciones entre diferentes atributos del dataset es la columna que indica la longitud de los textos:

In [22]:
tweets['length'] = tweets['text'].str.len()
tweets.head()

,id,keyword,location,text,target,length
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88


#### Localización de los tweets
La columna location indica la ubicación desde donde fue mandado el tweet. 
Este campo puede ser nulo, por eso se crea una columna booleana indicando si el campo existe o no.
Para mayor practicidad se crea una nueva columna del tipo integer que indique si el tweet tiene información de localización (1) o no (0):

In [23]:
tweets['located'] = tweets['location'].notnull().astype(int)
tweets.head()

,id,keyword,location,text,target,length,located
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,0


### Palabra clave de los tweets.
La columna 'keyword' contiene una palabra clave por cada tweet del dataset.
Este campo, al igual que 'location', puede ser nulo, así que por las mismas razones que con el campo 'location' se crea una columna booleana (tipo integer) que indique si el tweet posee palabras claves (1) o no(0):

In [24]:
tweets['has_keyword'] = tweets['keyword'].notnull().astype(int)
tweets.head()

,id,keyword,location,text,target,length,located,has_keyword
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,0,0


### Longitud de la palabra clave de los tweets
Las palabras claves de los tweets son texto, entonces es de interés la longitud de la palabra clave de los tweets:

In [26]:
tweets['keyword_length'] = tweets['keyword'].str.len()
tweets['keyword_length'].fillna(0, inplace=True)
tweets.head()

,id,keyword,location,text,target,length,located,has_keyword,keyword_length
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,0,0,0.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,0,0,0.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,0,0,0.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,0,0,0.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,0,0,0.0


### Longitud total del tweet
Un tweet esta compuesto por su texto y una palabra clave asociada a este texto, entonces se realiza la suma de ambas longitudes de los textos y se guarda en una nueva columna:

In [27]:
tweets['tweet_length'] = tweets['length'] + tweets['keyword_length']
tweets.head()

,id,keyword,location,text,target,length,located,has_keyword,keyword_length,tweet_length
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,0,0,0.0,69.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,0,0,0.0,38.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,0,0,0.0,133.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,0,0,0.0,65.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,0,0,0.0,88.0
